In [1]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\shabh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
print(pyodbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'ODBC Driver 18 for SQL Server', 'MySQL ODBC 9.6 ANSI Driver', 'MySQL ODBC 9.6 Unicode Driver', 'ODBC Driver 17 for SQL Server']


In [3]:
def fetch_data_from_sql():
    conn_str = (
       "Driver={MySQL ODBC 9.6 Unicode Driver};"
       "Server=localhost;"
       "Database=ma;"
        "USER=root;"
        "PASSWORD=Shabari@128;"
        "PORT=3306;"
    )
    conn = pyodbc.connect(conn_str)
    query = "SELECT Review_ID, Customer_ID, Product_ID, Review_date, Product_Rating, Review_Text FROM customer_review_fact"
    df = pd.read_sql(query, conn)
    conn.close()
    return df
customer_reviews_df = fetch_data_from_sql()
customer_reviews_df.head()

C:\Users\shabh\AppData\Local\Temp\ipykernel_9896\1369414443.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Review_ID,Customer_ID,Product_ID,Review_date,Product_Rating,Review_Text
0,R415901,C100584,P1009,31-12-2025,4,Very good product.
1,R336157,C100411,P1006,28-12-2025,1,Extremely poor quality.
2,R987386,C100408,P1001,25-12-2025,1,Very disappointed with this purchase.
3,R957334,C100064,P1004,22-12-2025,3,"Average quality, acceptable."
4,R516153,C101131,P1009,19-12-2025,5,"Excellent quality, highly recommend."


In [4]:
sia = SentimentIntensityAnalyzer()
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

In [5]:
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'


In [6]:
def sentiment_bucket(score):
    if  score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

In [7]:
customer_reviews_df['SentimentScore'] = customer_reviews_df['Review_Text'].apply(calculate_sentiment) 
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Product_Rating']), axis=1) 
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)
print(customer_reviews_df.head())


  Review_ID Customer_ID Product_ID Review_date  Product_Rating  \
0   R415901     C100584      P1009  31-12-2025               4   
1   R336157     C100411      P1006  28-12-2025               1   
2   R987386     C100408      P1001  25-12-2025               1   
3   R957334     C100064      P1004  22-12-2025               3   
4   R516153     C101131      P1009  19-12-2025               5   

                             Review_Text  SentimentScore SentimentCategory  \
0                     Very good product.          0.4927          Positive   
1                Extremely poor quality.         -0.5256          Negative   
2  Very disappointed with this purchase.         -0.5256          Negative   
3           Average quality, acceptable.          0.3182    Mixed Positive   
4   Excellent quality, highly recommend.          0.7574          Positive   

  SentimentBucket  
0     0.0 to 0.49  
1    -1.0 to -0.5  
2    -1.0 to -0.5  
3     0.0 to 0.49  
4      0.5 to 1.0  


In [9]:
customer_reviews_df.to_csv('Customer_reviews_with_Sentiments.csv', index=False)